In [ ]:
# Install a pip package in the current Jupyter kernel - https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/
import sys
!{sys.executable} -m pip install matplotlib numpy

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Specify the path to your JSON file
json_file_path = '/Users/bthj/QD/conf-static_mutation_rate_combinations_-_delete_rates/evolution-run-analysis_qd-scores,genome-statistics_step-1000_1684831116850.json'

# Specify the constant multiplier for x-values
x_multiplier = 1000  # Set this value as the step size in the JSON file name

# Read data from JSON file
def read_data_from_json(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

# Read data from JSON file
data = read_data_from_json(json_file_path)

# Extract data arrays from JSON
def extract_data_arrays(data, attribute):
    arrays = []
    for evo_run in data['evoRuns']:
        iterations = evo_run['iterations']
        if iterations and len(iterations) > 0:
            for iteration in iterations:
                if attribute in iteration:
                    iteration["label"] = evo_run["label"]
                    arrays.append({
                        "dataArray": iteration[attribute],
                        "label": evo_run["label"]   
                    })
    return arrays

# Render graph for each array
def render_graphs(arrays, x_multiplier, y_label):
    num_arrays = len(arrays)
    num_cols = 2  # Number of columns in the grid (you can adjust this)
    num_rows = (num_arrays + num_cols - 1) // num_cols
    fig, axs = plt.subplots(num_rows, num_cols, figsize=(12, 20))
    fig.tight_layout(pad=3.0)

    for i, arrayContainer in enumerate(arrays):
        array = arrayContainer["dataArray"]
        arrayLabel = arrayContainer["label"]
        x_values = np.arange(len(array)) * x_multiplier
        row = i // num_cols
        col = i % num_cols
        ax = axs[row, col] if num_rows > 1 else axs[col]

        if all(isinstance(element, (int, float)) for element in array):
            # Array contains numbers
            ax.plot(x_values, array)
        elif all(isinstance(element, dict) for element in array):
            # Array contains objects
            attributes = array[0].keys()  # Assuming all objects have the same attributes
            for attribute in attributes:
                values = [obj[attribute] for obj in array]
                ax.plot(x_values, values, label=attribute)
            ax.legend()

        ax.set_title(arrayLabel, fontsize=10, pad=10)
        plt.xlabel('Iteration')
        plt.ylabel(y_label)

    # Remove empty subplots
    for i in range(num_arrays, num_rows * num_cols):
        row = i // num_cols
        col = i % num_cols
        axs[row, col].axis('off')

    plt.show()

Plot QD scores for each evolution run:

In [ ]:

# Extract data arrays from JSON
qd_score_arrays = extract_data_arrays(data, 'qdScores')

# Render graph for each array with multiplied x-values
render_graphs(qd_score_arrays, x_multiplier, 'QD score')


Plot genome statistics (averages of node and connection counts):

In [ ]:
# Extract data arrays from JSON
genome_statistics_arrays = extract_data_arrays(data, 'genomeStatistics')

# Render graph for each array with multiplied x-values
render_graphs(genome_statistics_arrays, x_multiplier, 'Genome stats')